<a href="https://colab.research.google.com/github/pranavkarnani/StoryGenerator/blob/Vanilla-GPT-2/GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! pip install transformers

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
import torch.nn.functional as F
import torch.nn as nn
import csv

In [ ]:
import pandas as pd
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import Dataset, random_split, DataLoader, RandomSampler, SequentialSampler

In [ ]:
from tqdm.auto import tqdm

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
RANDOM_SEED = 73
BATCH_SIZE = 1

EPOCHS = 4
SAMPLE_EVERY = 10000

MAX_INPUT_SEQUENCE_LENGTH = 600

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

special_tokens_dict = {'bos_token': '<BOS>', 'eos_token': '<EOS>', 'pad_token': '<PAD>', 'sep_token': '<SEP>'}
num_added_tokens = tokenizer.add_special_tokens(special_tokens_dict)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/refined.csv")

In [ ]:
len(tokenizer)

In [ ]:
data.loc[0, 'storyline']

In [ ]:
class StoryOutlineDataset(Dataset):

    def __init__(self, data, tokenizer, max_input_length):

        self.tokenizer = tokenizer
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        self.data = data
        self.labels_attn = []

        for i in tqdm(range(len(self.data))):
            text = self.data.loc[i, 'text']
            outline = self.data.loc[i, 'storyline'].split(' ')
            outline = " ".join(outline[:100]).replace("<SEP>", "")

            input = outline + "<SEP>" + text

            encodings_dict_story = tokenizer('<BOS> ' + input + ' <EOS>',
                                     truncation=True,
                                     max_length=max_input_length,
                                     padding=True
                                    )
            
            self.input_ids.append(torch.tensor(encodings_dict_story['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict_story['attention_mask']))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, ind):
        return self.input_ids[ind], self.attn_masks[ind]

In [ ]:
story_dataset = StoryOutlineDataset(data.loc[0:50000], tokenizer, MAX_INPUT_SEQUENCE_LENGTH)

In [ ]:
from torch.utils.data import random_split

In [ ]:
def train_val_split(split, dataset):
    train_size = int(split * len(dataset))
    val_size = len(dataset) - train_size
    return train_size, val_size

In [ ]:
train_size, val_size = train_val_split(0.8, story_dataset)
train_dataset, val_dataset = random_split(story_dataset, [train_size, val_size])

In [ ]:
torch.cuda.manual_seed_all(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True)

In [ ]:
learning_rate = 5e-4
eps = 1e-8
warmup_steps = 100

In [ ]:
tokenizer.encode("<SEP>")

In [ ]:
configuration = GPT2Config(vocab_size=len(tokenizer), n_positions = MAX_INPUT_SEQUENCE_LENGTH, 
                           activation_function = "gelu_new", resid_pdrop = 0.1, embd_pdrop = 0.2,
                           attn_pdrop = 0.2, output_attentions = True, output_hidden_states = True)

model_config = configuration.from_pretrained('gpt2', output_hidden_states=True)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config = model_config
model.resize_token_embeddings(len(tokenizer))

In [ ]:
import time
import datetime
scaler = torch.cuda.amp.GradScaler()

In [ ]:
def format_time(elapsed):
    return str(datetime.timedelta(seconds=int(round((elapsed)))))

In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/model.pth'))

In [ ]:
# model.resize_token_embeddings(len(tokenizer))

model.cuda()
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=eps)
total_steps = len(train_loader) * EPOCHS
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0 = 100,
                                                                 T_mult = 3,
                                                                 eta_min = 1e-7)

In [ ]:
mse_loss = nn.MSELoss()

In [ ]:
def format_out_texts(text):
    t_map = tokenizer.special_tokens_map
    for key in t_map:
        text = text.replace(t_map[key], '')
    return text

def inference(val_loader):
    model.eval()

    for i, batch in enumerate(val_loader):

        if i % 100 == 0:
            lens = np.array([])
            input_ids = batch[0].numpy()
            attn_masks = batch[1].numpy()

            truncated_input = []
            truncated_attention_mask = []
            for i, input_id in enumerate(input_ids):
                context_index = np.where(input_id == 50260)[0][0]
                truncated_input.append(input_id[:context_index+1])
                truncated_attention_mask.append(attn_masks[i][:context_index+1])
                lens = np.append(lens, context_index+1)
    
            max_len = int(np.amax(lens))

            padded_tokens = []
            for tok_ids in truncated_input:
                
                padded_tokens.append(list(tok_ids) + [0] * (max_len - len(tok_ids)))
                
            padded_tokens = torch.LongTensor(padded_tokens).to(device)
            attn_mask = np.zeros(padded_tokens.shape)
            
            for ix, lengths in enumerate(lens):
                print(ix)
                print(lengths)
                attn_mask[ix][:int(lengths)] = 1

            attn_mask = torch.tensor(attn_mask).long().to(device)

    story_ids = model.generate(padded_tokens, attention_mask=attn_mask,
                            num_beams=5,
                            max_length=800,
                            temperature=0.9,
                            remove_invalid_values = True,
                            top_k=50,
                            do_sample=True)

    raw_stories = [tokenizer.decode(story) for story in story_ids]
    output_texts = list(map(format_out_texts, raw_stories))
    print(output_texts)
    return output_texts

In [ ]:
def train(ep, train_loader):

    total_train_loss = 0

    for step, batch in enumerate(tqdm(train_loader)):

        model.train() 

        b_input_ids = batch[0]
        b_masks = batch[1].to(device)

        labels = b_input_ids.clone().numpy()

        for i, text in enumerate(b_input_ids.numpy()):
            context_index = np.where(text == 50260)[0][0]
            labels[i][:context_index+1] = -100

        model.zero_grad()

        b_input_ids = b_input_ids.to(device)
        labels = torch.tensor(labels).to(device)

        with torch.cuda.amp.autocast():
        
            outputs = model(b_input_ids,
                        attention_mask=b_masks,
                        labels = labels,
                        token_type_ids=None)

            
            loss = outputs[0]

            batch_loss = loss

            total_train_loss += batch_loss
            scaler.scale(batch_loss).backward() 
            scaler.step(optimizer) 
            scaler.update()
            scheduler.step()

    avg_train_loss = total_train_loss / len(train_loader)       

    print(f'Average Training Loss: {avg_train_loss}.')


def validate(val_dataloader, file_name):

    model.eval()
    total_eval_loss = 0

    for idx, batch in enumerate(val_dataloader):
        b_input_ids = batch[0]
        b_masks = batch[1].to(device)

        labels = b_input_ids.clone().numpy()

        for i, text in enumerate(b_input_ids.numpy()):
            context_index = np.where(text == 50260)[0][0]
            labels[i][:context_index+1] = -100

        b_input_ids = b_input_ids.to(device)
        labels = torch.tensor(labels).to(device)

        with torch.no_grad():        
            outputs  = model(b_input_ids,  
                            attention_mask=b_masks,
                            labels=labels)

            loss = outputs[0]

        batch_loss = loss
        total_eval_loss += batch_loss   

    avg_val_loss = total_eval_loss / len(val_dataloader)
    inference(val_dataloader)

    print(f'Validation loss: {avg_val_loss}.')
    torch.save(model.state_dict(), '/content/' + file_name)
    return model

In [ ]:
for epoch_i in range(0, EPOCHS):
    print(f'Epoch {epoch_i + 1} of {EPOCHS}')
    train(epoch_i, train_loader)
    # validate(val_loader, '/drive/MyDrive/model.pth')

In [ ]:
len(val_loader)
model.load_state_dict(torch.load('/content/drive/MyDrive/model.pth'))

In [ ]:
df = pd.DataFrame()

In [ ]:
model.cuda()

In [ ]:
master_input = []
master_output = []

def getOutputs(val_loader):
    model.eval()

    for i, batch in enumerate(val_loader):

        if i > 0:
            lens = np.array([])
            input_ids = batch[0].numpy()
            attn_masks = batch[1].numpy()

            truncated_input = []
            truncated_attention_mask = []
            for i, input_id in enumerate(input_ids):
                context_index = np.where(input_id == 50260)[0][0]
                truncated_input.append(input_id[:context_index+1])
                truncated_attention_mask.append(attn_masks[i][:context_index+1])
                lens = np.append(lens, context_index+1)
    
            max_len = int(np.amax(lens))
            master_input.append(truncated_input)
            padded_tokens = []
            for tok_ids in truncated_input:
                padded_tokens.append(list(tok_ids) + [0] * (max_len - len(tok_ids)))
                
            padded_tokens = torch.LongTensor(padded_tokens).to(device)
            attn_mask = np.zeros(padded_tokens.shape)
            
            for ix, lengths in enumerate(lens):
                attn_mask[ix][:int(lengths)] = 1

            attn_mask = torch.tensor(attn_mask).long().to(device)

            story_ids = model.generate(padded_tokens, attention_mask=attn_mask,
                                num_beams=5,
                                max_length=800,
                                temperature=1,
                                remove_invalid_values = True,
                                top_k=50,
                                do_sample=True)

            raw_stories = [tokenizer.decode(story) for story in story_ids]
            master_output.append(raw_stories)

    return master_output

In [ ]:
stories = getOutputs(val_loader)

In [ ]:
df['input'] = mmInput
df['output'] = master_input

In [ ]:
df.to_csv('GPT2_Stories.csv')